In [8]:
import pandas as pd
import numpy as np
import os
import scipy.stats as stats

In [11]:
new_notes = pd.read_excel("path\\new_notes_5.xlsx") #read the main file

In [12]:
new_notes["bin_bad"] = np.nan # See the bad files in bin and multi methods
new_notes["mul_bad"] = np.nan

In [13]:
mul = new_notes["bin_mean"].isna() #If the files are there then it a "Y" if not - "N"
for i in range(len(mul)):
    if mul[i] == True:
        new_notes["bin_bad"].iloc[i]  = "Y"
    else:
        new_notes["bin_bad"].iloc[i] = "N"

<ipython-input-13-17182e5f5b83>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_notes["bin_bad"].iloc[i] = "N"


In [14]:
mul = new_notes["mul_mean"].isna()
for i in range(len(mul)):
    if mul[i] == True:
        new_notes["mul_bad"].iloc[i]  = "Y"
    else:
        new_notes["mul_bad"].iloc[i] = "N"

<ipython-input-14-9305ce120e44>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_notes["mul_bad"].iloc[i] = "N"


In [33]:
path_noise = ("noise path") #defind path to read noise files
path_EDA = ("EDA path") #define path to read EDA files
clean_bin = ("bin path") #define path to noise-clean EDA's bin
clean_mul = ("mul path") #define path to noise-clean EDA's mul

In [34]:
def clean_bin_func(clean_bin): #clean the bin files
    for row in new_notes[new_notes["bin_bad"] == "Y"].iterrows():
        for item in os.listdir(path_EDA):
            index = item.find("_") 
            sub_number = item[:index]
            if row[1][3] == int(sub_number):
                for item_N in os.listdir(path_noise):
                    if item_N.startswith(sub_number+"_") and item_N.endswith("Binary.csv"):
                        noise= pd.read_csv(path_noise+"\\"+item_N) #read the right noise
                        EDA = pd.read_csv(path_EDA+"\\"+item) #read the right EDA file
                    
                        noise["StartTime"] =  pd.to_datetime(noise["StartTime"], dayfirst=True) #convert the start time of the noise to datetime (because of the format it reading days and month in the oposite way so we need dayfirst=true)
                        noise["EndTime"] =  pd.to_datetime(noise["EndTime"], dayfirst=True) #convert the end time of the noise to datetime
                        EDA["GMT date"] = pd.to_datetime(EDA["GMT date"]) #convert the EDA (GMT) time to datetime
                        total_row_length=len(EDA.index) #count the total number of rows

                        noisy_parts = noise[noise["BinaryLabels"] ==-1][["StartTime", "EndTime"]] #search for the noisy parts
                        artifacts_length = int(len(noisy_parts.index)) #count how much "-1" ("bad signal") the file has
                        set_times = set() #make an empty set
                        for row in EDA.loc[2:,"GMT date"]: #for every row in the EDA file:
                            for index,i in noisy_parts.iterrows(): #check if the row in the EDA file is between the start and the end time of the noise
                                if i["StartTime"]<=row<=i["EndTime"]:
                                    set_times.add(row) #if it does then add the 'bad' rows to the set

                        bool_list = list() #make an emty list which will contain the true(good) and false(bad) rows of the EDA file
                        for row in EDA.iterrows(): #for each row in the EDA file 
                            boolean = (row[1][3] not in set_times) #decise whether it is not in the 'bad' rows (1,3 because itterrows gives tuple of index and row with all of the columns)
                            bool_list.append(boolean) #add the trues and false to the empty list
                        clean_EDA = EDA.loc[np.array(bool_list)].drop(["Unnamed: 0","Unnamed: 0.1"], axis=1) #make a clean EDA file by subsetting the trues and false


                        target_path = os.path.join(clean_bin, item) #make a target path for the new clean EDA file
                        clean_EDA.to_csv(target_path)            #write the file

In [9]:
def clean_mul_func(clean_mul):
    for row in new_notes[new_notes["mul_bad"] == "Y"].iterrows():
        for item in os.listdir(path_EDA):
            index = item.find("_") 
            sub_number = item[:index]
            if row[1][3] == int(sub_number):
                for item_N in os.listdir(path_noise):
                    if item_N.startswith(sub_number+"_") and item_N.endswith("Multiclass.csv"):
                        noise= pd.read_csv(path_noise+"\\"+item_N) #read the right noise
                        EDA = pd.read_csv(path_EDA+"\\"+item) #read the right EDA file

                        noise["StartTime"] =  pd.to_datetime(noise["StartTime"], dayfirst=True) #convert the start time of the noise to datetime (because of the format it reading days and month in the oposite way so we need dayfirst=true)
                        noise["EndTime"] =  pd.to_datetime(noise["EndTime"], dayfirst=True) #convert the end time of the noise to datetime
                        EDA["GMT date"] = pd.to_datetime(EDA["GMT date"]) #convert the EDA (GMT) time to datetime
                        total_row_length=len(EDA.index) #count the total number of rows

                        noisy_parts = noise[noise["MulticlassLabels"] ==-1][["StartTime", "EndTime"]] #search for the noisy parts
                        artifacts_length = int(len(noisy_parts.index)) #count how much "-1" ("bad signal") the file has
                        set_times = set() #make an empty set
                        for row in EDA.loc[2:,"GMT date"]: #for every row in the EDA file:
                            for index,i in noisy_parts.iterrows(): #check if the row in the EDA file is between the start and the end time of the noise
                                if i["StartTime"]<=row<=i["EndTime"]:
                                    set_times.add(row) #if it does then add the 'bad' rows to the set

                        bool_list = list() #make an emty list which will contain the true(good) and false(bad) rows of the EDA file
                        for row in EDA.iterrows(): #for each row in the EDA file 
                            boolean = (row[1][3] not in set_times) #decise whether it is not in the 'bad' rows (1,3 because itterrows gives tuple of index and row with all of the columns)
                            bool_list.append(boolean) #add the trues and false to the empty list
                        clean_EDA = EDA.loc[np.array(bool_list)].drop(["Unnamed: 0","Unnamed: 0.1"], axis=1) #make a clean EDA file by subsetting the trues and false


                        target_path = os.path.join(clean_mul, item) #make a target path for the new clean EDA file
                        clean_EDA.to_csv(target_path)      #write the file

In [36]:
clean_bin_func(clean_bin) #Call the function

In [16]:
clean_mul_func(clean_mul)

In [40]:
# Prepare for matlab

path_EDA = ("C:\\Users\\Tali\\Documents\\study\\work\\seminar\\sumer 2022\\Braclets raw\\more_then_25_error\\bin\\d")
path_new = ("C:\\Users\\Tali\\Documents\\study\\work\\seminar\\sumer 2022\\Braclets raw\\more_then_25_error\\bin\\d")
for item in os.listdir(path_EDA):
    EDA = pd.read_csv(path_EDA+"\\"+item) # read the EDA file
    data = EDA.iloc[2:, 1] # Take only the relevant data
    
    index = item.find(".") 
    only_name = item[:index] #Take the name of the file
    
    save_path = os.path.join(path_new, only_name) #Where to save the file
    
    np.savetxt(save_path + '.txt', data) #Save the current file as txt

In [1]:
# After matlab calculate SCL statistics and update it to the notes file

def calculations(source): # For binary SCL files
    for SCL in os.listdir(source):
        if "SCL" in SCL:
            Key = int(SCL.split("_")[0])
            file_path = os.path.join(source, SCL)
            excel = pd.read_excel(file_path)
            
            kurt = stats.kurtosis(excel.values.flatten(), bias=False)
            ske = stats.skew(excel.values.flatten(), bias=False)
            mean = excel.values.flatten().mean()
            median = np.median(excel.values.flatten())
            stde = excel.values.flatten().std()

            new_notes["bin_kurt"][new_notes["Subject number"] == Key] = kurt
            new_notes["bin_skew"][new_notes["Subject number"] == Key] = ske
            new_notes["bin_mean"][new_notes["Subject number"] == Key] = mean
            new_notes["bin_med"][new_notes["Subject number"] == Key] = median
            new_notes["bin_std"][new_notes["Subject number"] == Key] = stde

In [2]:
def calculations_mul(source): # For multiclass SCL files
    for SCL in os.listdir(source):
        if "SCL" in SCL:
            Key = int(SCL.split("_")[0])
            file_path = os.path.join(source, SCL)
            excel = pd.read_excel(file_path)
            
            kurt = stats.kurtosis(excel.values.flatten(), bias=False)
            ske = stats.skew(excel.values.flatten(), bias=False)
            mean = excel.values.flatten().mean()
            median = np.median(excel.values.flatten())
            stde = excel.values.flatten().std()

            new_notes["mul_kurt"][new_notes["Subject number"] == Key] = kurt
            new_notes["mul_skew"][new_notes["Subject number"] == Key] = ske
            new_notes["mul_mean"][new_notes["Subject number"] == Key] = mean
            new_notes["mul_med"][new_notes["Subject number"] == Key] = median
            new_notes["mul_std"][new_notes["Subject number"] == Key] = stde

In [43]:
source = "bin\mat path"

In [44]:
calculations(source) #calculation for binary SCL

<ipython-input-1-48e9f2acfa15>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_notes["bin_kurt"][new_notes["Subject number"] == Key] = kurt
<ipython-input-1-48e9f2acfa15>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_notes["bin_skew"][new_notes["Subject number"] == Key] = ske
<ipython-input-1-48e9f2acfa15>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_notes["bin_mean"][new_notes["Subject number"] == Key] = mean
<i

In [20]:
calculations_mul(source) #calculation for multiclass SCL

<ipython-input-2-e5316f59ffd8>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_notes["mul_kurt"][new_notes["Subject number"] == Key] = kurt
<ipython-input-2-e5316f59ffd8>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_notes["mul_skew"][new_notes["Subject number"] == Key] = ske
<ipython-input-2-e5316f59ffd8>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_notes["mul_mean"][new_notes["Subject number"] == Key] = mean
<i

<ipython-input-2-e5316f59ffd8>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_notes["mul_kurt"][new_notes["Subject number"] == Key] = kurt
<ipython-input-2-e5316f59ffd8>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_notes["mul_skew"][new_notes["Subject number"] == Key] = ske
<ipython-input-2-e5316f59ffd8>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_notes["mul_mean"][new_notes["Subject number"] == Key] = mean
<i

<ipython-input-2-e5316f59ffd8>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_notes["mul_kurt"][new_notes["Subject number"] == Key] = kurt
<ipython-input-2-e5316f59ffd8>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_notes["mul_skew"][new_notes["Subject number"] == Key] = ske
<ipython-input-2-e5316f59ffd8>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_notes["mul_mean"][new_notes["Subject number"] == Key] = mean
<i

In [50]:
# new_notes=new_notes.drop(["Unnamed: 0"], axis=1) #clean columns
new_notes.to_excel("Path\\new_notes_6.xlsx") #write the apdated main table